In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

This code loads your CSV file, splits the data into a training set and a test set, and creates a DataLoader for each. The DataLoader can be used to iterate through the data in batches, which is useful for training a neural network.

You can replace 'yourfile.csv' with the path to your actual file. Also, note that this assumes your CSV file doesn't have a header. If it does, you might need to skip the first row.

In [94]:

# Define custom dataset
class AccelDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [95]:
# Load CSV file
WORKAREA_PATH = 'C:\\Users\\fl4m1\\Documents\\School\\CURRENT\\2023-24 RCSA Research Assistantship\\Work Area\\Data'
dataframe = pd.read_csv(WORKAREA_PATH + '\\Week 1\\Left then Right\\Processed\\Type3-Freq10-Labeled_Motion-sessions_2023-08-26_17-35-54.csv')

# Get data and labels from dataframe
data = dataframe.iloc[:, :3].values  # x, y, z data
labels = dataframe.iloc[:, 3:].values  # labels

# Split data into training and test sets
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert data to tensors
data_train = torch.tensor(data_train, dtype=torch.float32)
data_test = torch.tensor(data_test, dtype=torch.float32)

# Convert labels to tensors
labels_train = torch.tensor(labels_train, dtype=torch.long)
labels_test = torch.tensor(labels_test, dtype=torch.long)

# Create data loaders
train_dataset = AccelDataset(data_train, labels_train)
test_dataset = AccelDataset(data_test, labels_test)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In this code, we added 2 more convolutional layers, which can extract more complex features from your accelerometer data. The number of output channels in the convolutional layers gradually increases, as it is common in many deep learning models to gradually increase the complexity and decrease the spatial size.

In [96]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Assuming input sequence length is 100 and each step has 3 dimensions (x, y, z)
        self.conv1 = nn.Conv1d(3, 64, kernel_size=5)  # Change the number of input channels to 3
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=5)
        #*
        #*NOTE FROM STEPHEN:
        #*    #why is 256 multiplied with 82? I can see that it is applied after flattening the features, but was 82 chosen based on our old number of output classes?
        #*
        self.fc1 = nn.Linear(256 * 82, 128) 
        self.fc2 = nn.Linear(128, 16)   # Output classes are 5 (left, right, forward, pushup, other)
        #will be changing the output classes to 10 (left_s, left_e, right_s, right_e, forward_s, forward_e, pushup_s, pushup_e, stationary, other)
        #will be changing to type 3 classification: {16 classes: Forward_k_s (1), Forward_k_e (2), Forward_t_s (3), Forward_t_e (4),
        #   Left_p_s (5), Left_p_e (6), Left_h_s (7), Left_h_e (8), Right_p_s (9), Right_p_e (10), Right_h_s (11), Right_h_e (12),
        #   Pushup_s (13), Pushup_e (14), Stationary (15), Other (16)}
        #k stands for knee, t stands for table, p stands for push, h stands for hold, s stands for start, and e stands for end

    def forward(self, x):
        print('---')
        print(x.shape)
        print('---')
        
        x = F.relu(self.conv1(x))
        print(x.shape)
        print('---')
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.num_flat_features(x))  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Apply softmax to the output layer

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



Please note that the dimension of the input to the fully connected layer depends on the output size of your last convolutional layer. This code assumes that after 3 layers of convolution with kernel size 5 and stride 1, the sequence length is reduced to 82 (from the original 100). You may need to adjust this according to your own situation.

In [97]:
# Instantiate the network and optimizer
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Assume we have a data loader `dataloader` which loads our accelerometer data
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        print(inputs.shape) # 64 x 3
        #inputs = inputs.reshape(1, 64, 3)
        #inputs = inputs.T
        outputs = net(inputs) ##error
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([64, 3])
---
torch.Size([3, 64, 1])
---


RuntimeError: Given groups=1, weight of size [64, 3, 5], expected input[3, 64, 1] to have 3 channels, but got 64 channels instead

In [ ]:
# Now we will validate the model using test data
correct = 0
total = 0

with torch.no_grad():   # Since we're not training, we don't need to calculate the gradients
    for data in test_dataloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))

---


AttributeError: 'Tensor' object has no attribute 'Shape'